In [1]:
import requests
from tqdm.auto import tqdm
import json
import pandas as pd
from datetime import datetime
import os


### Ideias de classificação

 Com base na descrição da pra vericar se houve mortos ou não

 Tem também o parâmetro crash, da pra usar ele como alguma coisa na hora de classificar

 Da pra criar com base nos components uma classificação voltada para o texto em summary.



## 1. Aquisição e Pré-processamento de Dados

### Objetivo

* Nesta seção, detalhamos o processo de aquisição e pré-processamento dos dados de reclamações de veículos da NHTSA. O objetivo foi preparar os dados para treinamento de um modelo de classificação baseado em embeddings.

### Aquisição dos dados

* Os dados foram obtidos através da API fornecida do endereço `https://api.nhtsa.gov`

* Com o uso da biblioteca requests, do Python, foi possível adquirir as informações acerca das reclamações de veículos do NHTSA

In [2]:
def get_all_model_years(output_file: str = "model_years.json") -> None:
    """
    Obtem todos o ano de todos os modelos dispoíveis.
    """
    url = "https://api.nhtsa.gov/products/vehicle/modelYears?issueType=c"
    try:
        response = requests.get(url)
        results = response.json().get('results')
        if results:
            with open(output_file, "w") as file:
                model_year = [year['modelYear'] for year in results]
                json.dump(model_year, file, indent=4)
                print(f"Dados salvos em {output_file}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

In [3]:
def get_all_makes_model_year(list_model_year: list[dict], output_file: str = "make_modelyear.json") -> None:
    """
    A partir da lista de anos dos modelos disponíveis, obtém a marca daquele modelo e salva os dados em um arquivo JSON a cada 100 registros.
    """
    url = "https://api.nhtsa.gov/products/vehicle/makes?modelYear={model_year}&issueType=c"
    make_modelyear = []
    record_count = 0

    for model_year in tqdm(list_model_year, desc="Capturando modelo e ano", colour="green"):
        try:
            response = requests.get(url.format(model_year=model_year))
            response.raise_for_status()
            results = response.json().get('results', [])
            
            if results:
                make_modelyear.extend(results)
                record_count += len(results)
                
                if record_count >= 100:
                    with open(output_file, "w") as file:
                        json.dump(make_modelyear, file, indent=4)
                    record_count = 0
                    
        except Exception as e:
            print(f"Ocorreu um erro ao processar o ano {model_year}: {e}")
            continue
        
    if make_modelyear:
        with open(output_file, "w") as file:
            json.dump(make_modelyear, file, indent=4)

    print(f"Dados salvos em {output_file}")

In [4]:
def get_all_models_make_year(list_make_year: list[dict], output_file: str = "model_make_year.json") -> None:
    """
    Obtém uma lista de dicionários que contém a marca, o modelo e o ano para cada carro, 
    salvando os dados em um arquivo JSON a cada 100 registros.
    """
    url = "https://api.nhtsa.gov/products/vehicle/models?modelYear={modelYear}&make={make}&issueType=c"
    model_make_year = []
    record_count = 0

    for item in tqdm(list_make_year, desc="Capturando marca, modelo e ano", colour="yellow"):
        try:
            response = requests.get(url.format(modelYear=item.get('modelYear'), make=item.get("make")))
            response.raise_for_status()
            results = response.json().get('results', [])

            if results:
                model_make_year.extend(results)
                record_count += len(results)
                
                if record_count >= 100:
                    with open(output_file, "w") as file:
                        json.dump(model_make_year, file, indent=4)
                    record_count = 0
        except Exception as e:
            print(f"Ocorreu um erro ao processar o item {item}: {e}")
            continue

    if model_make_year:
        with open(output_file, "w") as file:
            json.dump(model_make_year, file, indent=4)

    print(f"Dados salvos em {output_file}")

In [5]:
def get_all_complaints(make_model_year: list[dict], output_file: str = "complaints.json") -> list[dict]:
    """
    Obtém uma lista de dicionários contendo as reclamações por modelo, marca e ano.
    Os dados são salvos em um arquivo JSON a cada 100 registros processados.
    """
    url_template = "https://api.nhtsa.gov/complaints/complaintsByVehicle?make={make}&model={model}&modelYear={modelYear}"
    complaints = []
    record_count = 0

    for item in tqdm(make_model_year, desc="Capturando as reclamações", colour="blue"):
        try:
            url = url_template.format(make=item.get('make'), model=item.get('model'), modelYear=item.get('modelYear'))
            response = requests.get(url)
            response.raise_for_status()
            results = response.json().get('results', [])
            for result in results:
                complaints.append(result)
                record_count += 1

                if record_count >= 100:
                    with open(output_file, "w") as file:
                        json.dump(complaints, file, indent=4)
                        record_count = 0
        except Exception as e:
            print(e)
            continue
            
    if complaints:
        with open(output_file, "w") as file:
            json.dump(complaints, file, indent=4)

    print(f"Dados salvos em {output_file}")
    return complaints

In [6]:
BASE_DIR = "../../data/json"

In [ ]:
get_all_model_years(output_file=os.path.join(BASE_DIR, "model_years.json"))
model_year = json.load(open(os.path.join(BASE_DIR, "model_years.json"), "r"))

In [ ]:
get_all_makes_model_year(model_year, output_file=os.path.join(BASE_DIR, "make_modelyear.json"))
make_modelyear = json.load(open(os.path.join(BASE_DIR, "make_modelyear.json"), "r"))

In [ ]:
get_all_models_make_year(make_modelyear, output_file=os.path.join(BASE_DIR, "model_make_year.json"))
model_make_year = json.load(open(os.path.join(BASE_DIR, "model_make_year.json"), "r"))

In [ ]:
# Dropando duplicados
df_model_make_year = pd.DataFrame(model_make_year)
df_model_make_year.drop_duplicates(inplace=True)
df_model_make_year.dropna(inplace=True)
model_make_year = df_model_make_year.to_dict(orient='records')
model_make_year

In [54]:
json.dump(model_make_year, open(os.path.join(BASE_DIR, "model_make_year.json"), "w"), indent=4, ensure_ascii=False)

#### Obtendo todas as reclamações

In [7]:
model_make_year = json.load(open(os.path.join(BASE_DIR, "model_make_year.json"), 'r'))

In [ ]:
get_all_complaints(model_make_year, output_file=os.path.join(BASE_DIR, "complaints.json"))